In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re as re
import nltk as nl
from nltk import word_tokenize, pos_tag, bigrams
from nltk.parse.api import ParserI

sns.set
pd.options.display.max_colwidth = 200

In [2]:
data_raw = pd.read_csv('C:\\Users\\DELL\\Desktop\\Akshay\\Datasets\\factoid_questions_classification\\factoid_questions_dataset.txt', sep='delimiter', header=None, names=['Factoids'], dtype={'Factoids':str})

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
data_raw

,Factoids
0,DESC:manner How did serfdom develop in and then leave Russia ?
1,ENTY:cremat What films featured the character Popeye Doyle ?
2,DESC:manner How can I find a list of celebrities ' real names ?
3,ENTY:animal What fowl grabs the spotlight after the Chinese Year of the Monkey ?
4,ABBR:exp What is the full form of .com ?
...,...
995,DESC:manner How do you look up criminal records on the Internet ?
996,"LOC:other Where is the actress , Marion Davies , buried ?"
997,DESC:desc What is an example of an actual case of using the Fourteenth Amendment ?
998,HUM:ind Who was the Charlie perfume woman ?


In [7]:
data_raw['Factoids'] = data_raw.Factoids.str.replace(':', ' ')
data_raw['Class'] = data_raw.Factoids.str.split('\s+').str[0]
data_raw['Sub-Class'] = data_raw.Factoids.str.split('\s+').str[1]
data_raw['Questions'] = data_raw.Factoids.str.split().str[2:].str.join(' ')
data_raw.drop('Factoids', axis=1)

,Class,Sub-Class,Questions
0,DESC,manner,How did serfdom develop in and then leave Russia ?
1,ENTY,cremat,What films featured the character Popeye Doyle ?
2,DESC,manner,How can I find a list of celebrities ' real names ?
3,ENTY,animal,What fowl grabs the spotlight after the Chinese Year of the Monkey ?
4,ABBR,exp,What is the full form of .com ?
...,...,...,...
995,DESC,manner,How do you look up criminal records on the Internet ?
996,LOC,other,"Where is the actress , Marion Davies , buried ?"
997,DESC,desc,What is an example of an actual case of using the Fourteenth Amendment ?
998,HUM,ind,Who was the Charlie perfume woman ?


In [68]:
max_len = 37

text = "How far is it from Denver to Aspen?"

tok_text = word_tokenize(text)
print(f"Tokens: {tok_text}")

postag_text = pos_tag(tok_text)
print(f"Pos Tags: {list(postag_text)}")

t = tok_text+["<PAD>"]*(37-len(tok_text)) if len(tok_text) < 37 else tok_text
print(f"Padded Text: {t}")

Tokens: ['How', 'far', 'is', 'it', 'from', 'Denver', 'to', 'Aspen', '?']
Pos Tags: [('How', 'WRB'), ('far', 'RB'), ('is', 'VBZ'), ('it', 'PRP'), ('from', 'IN'), ('Denver', 'NNP'), ('to', 'TO'), ('Aspen', 'VB'), ('?', '.')]
Padded Text: ['How', 'far', 'is', 'it', 'from', 'Denver', 'to', 'Aspen', '?', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']


In [20]:
toks = [word_tokenize(question) for question in data_raw.Questions]

In [93]:
postagged = [pos_tag(tok) for tok in toks]

pos_tags = []

for i in range(len(postagged)):
    postagged[i]
    tags = []
    for j in range(len(postagged[i])):
        tags.append(postagged[i][j][1])
    pos_tags.append(tags)

def list_to_array_pos_tag(pos_tag):
    pos_tags = pos_tag+["<PAD>"]*(37-len(pos_tag)) if len(pos_tag) < 37 else pos_tag
    return np.array([
        '-' if pos_tag == "<PAD>" else pos_tag
        for pos_tag in pos_tags
    ])

In [95]:
list_to_array_pos_tag(pos_tags[0])

array(['WRB', 'VBD', 'VB', 'VB', 'IN', 'CC', 'RB', 'VB', 'NNP', '.', '-',
       '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
       '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'],
      dtype='<U3')

In [20]:
features = []
for i in range(len(questions)):
    postagged[i]
    
    question_features = np.array(())
    question_count = []
    token_count = []
    tokens = []
    postags = []
    
    for j in range(len(postagged[i])):
        question_count.append(i)
        token_count.append(j)
        tokens.append(postagged[i][j][0])
        postags.append(postagged[i][j][1])
    
    a = np.array(question_count)
    b = np.array(token_count)
    c = np.array(tokens)
    d = np.array(postags)
    
    question_features = np.column_stack((a,b,c,d))
    features.append(question_features)

features = np.array(features)